# Even more velocity stuff

I came up with a method for assigning velocities to events like notes. At first I was thinking of designing it like automation curves consisting of a set of points and interpolation. While that's a fun idea I still want to explore (particularly the interpolation idea), I decided on making another "functional" object, where you supply a "subdivision hierarchy", velocities to map to each subdivision level, a "time resolution", and an offset value.

Like onset functions or pitch functions, the idea is that you specify a cyclic pattern, and then you can call (evaluate) it like a function at whatever inputs you want. This, I feel, is a very flexible approach.

## Refining my approach

I'm currently using integer partitions to describe subdivision trees.

While I do think this is intuitive, I also think there's potential for a higher level of consistency. I really want to lean into this functional approach. I like having families of functions - pitch functions, onset functions, velocity functions, etc.

So, how would you go about using onset functions instead of lists of integer partitions?

## What if onset functions had a "subdivide" method?

Like, ok, what if onset functions had a method that let you subdivide them?

Say you have the following onset function:


In [ ]:
from harmonica.time._onsetfunc import OnsetFunc
from harmonica.utility._mixed import Mixed


ofunc = OnsetFunc([Mixed(x) for x in ["1/2", 2, 3]])

What if you wanted to subdivide it? I guess the question is, how would you specify the subdivision?

Let's take our example from yesterday, and express `[[6, 4], [3, 3, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]` as a list of onset functions.

In [2]:
subdiv_tree = [
    OnsetFunc([Mixed(x) for x in [6, 10]]),
    OnsetFunc([Mixed(x) for x in [3, 6, 8, 10]]),
    OnsetFunc([Mixed(x) for x in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]),
]

At this point, I think it's worth asking the question:

Why not just do what I've currently been doing? Why use onset functions, beyond it being pleasing to have consistency?

I don't have answers for that right now, so I'm just gonna put this idea on the backburner until motivation arises.

## Quick improvement

I currently have the duration (in units) as a field of the VelocityFunc class. This seems redundant, because that information can simply be derived from the subdivision tree. For this reason, I'm going to delete the duration field, and add a `dur_in_units` property.

## What about building velocity functions like I do onset functions?

With rhythm tools, it's useful to have means for building them up. That's why I added shifting, stretching, truncating, and concatenating in onset functions.

I would like to be able to do the same thing with velocity functions. 

- Shifting

This is trivial. Modifies the offset.

- Stretching

I believe this is also pretty trivial, because you need only modify the time resolution.

- Truncating

This is a bit more involved. You have to modify the subdivision tree. Like, if the subdivision tree is `[[6, 4], [3, 3, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]` and the time resolution is 1/4, and you want to truncate the pattern to a length of 2 beats, you basically have to sum over each integer in each subdivision level, and cut off the rest of the partition once it amounts to a duration greater than the truncation amount.

**A note**: For things like evaluating and truncating and what not - anything where you have to input a value in beats - I've been "quantizing" the input values to the time resolution, to make things simpler. It makes sense to do, because it's the grid resolution. Quantizing makes sense!

But, it is worth acknowledging that there are alternative techniques for, say, truncating a velocity function to a duration that isn't a multiple of its resolution. You would find the LCM (or something like that) of the truncation value and the time resolution, generating an entirely different resolution & subdivision tree. I will not be doing this though. For now, I am simplifying things with the assumption that we are quantizing & preserving the time resolution, unless it's explicitly changed by stretching for example.

- Concatenating

This one is pretty simple if the two velocity functions have the same time resolution and the same number of subdivision levels. In that case, you're just stitching together corresponding subdivision levels from each function's subdivision trees. For example, if you stitch `[[3,2], [1,1,1,1,1]]` and `[[4,2], [1,1,1,1,1,1]`, you get `[[3,2,4,2], [1,1,1,1,1,1,1,1,1,1,1]]`.

### Trying out the truncation algo

Let's give this a think.

In [4]:
from math import floor


def truncate(subdivs, res, new_dur_in_beats: Mixed):
    """Truncates the subdivision tree to a new duration (in beats)."""

    new_dur_in_beats = floor(new_dur_in_beats / res) * res

    new_dur_units = int(new_dur_in_beats / res)

    new_subdiv_tree = []

    for subdiv in subdivs:
        new_subdiv = []

        for i in subdiv:
            new_sum = sum(new_subdiv + [i])
            if new_sum > new_dur_units:
                diff = new_sum - new_dur_units
                if i - diff > 0:
                    new_subdiv.append(i - diff)
                break
            else:
                new_subdiv.append(i)

        new_subdiv_tree.append(new_subdiv)

    return new_subdiv_tree


tree = [[6, 4], [3, 3, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
print(truncate(tree, Mixed("1/4"), Mixed("7/3")))

[[6, 3], [3, 3, 2, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]


I think I've got it! Yeah!

## Decision on implementing concatenation

To keep things simple, if you concatenate two velocity functions, they must both have the same time resolution and subdivision depth.

There are still some questions left unanswered. Namely:

- How do we handle offsets?

and

- How do we handle velocity values?

The second one is much more troublesome. We have to choose between one set of velocity values or the other. I think we should just keep the current function's velocities and ignore the other's.

This does raise an interesting question. What if we don't keep the velocity values in our state? What if we just pass them as an argument when evaluating? Worth considering...

The offset question is the trickiest, I feel. 

### The Offset Problem of Velocity Function Concatenation

This is proving to be very fuckin' weird. I drew it out on paper, and yes, you can definitely concatenate two velocity functions. But designing the algorithm is the weird part.

Of course, if the offsets are both 0, it's incredibly trivial. You just stitch the corresponding levels of each subdivision tree together.

But when they have offsets, that's when things get REALLY weird - at least, in this representation they sure do.

So, if you have a subdivision that's like (6,4), and you have an offset of 1, well, shit's kind of fuckin' weird, right? Because you have to imagine that the 6 is split, such that at the start of the pattern, you get the last 5 units, then 4 comes after it, and then you get the first unit at the end. So it's like the (6,4) becomes (5,4,1). If the offset were 2, then the (6,4) becomes (4,4,2). 

Does that make sense? I sure hope so.

So, anyways, if you're stitching together these subdivision patterns, you have to have a way of marking these parts as either "open or close", right? We can borrow interval notation from math (open intervals being parens, and closed being brackets.)

So, with an offset of 2, [6,4] becomes (4,4,2). Then, if the other tree is [3,4], but with an offset of 2, it becomes (1,4,2).

So, stitching these together. Here's what you do. 

If either the last part of the first subdiv or the first part of the second subdiv are closed, then we just concatenate the two lists...

Oh! It's just occurred to me: a subdivision list is either open or closed, there's never any case where just the start is open or just the end is open. Furthermore, if either or both subdivision lists are closed, then you concatenate the whole thing like it's closed. Just a simple stitching together, in that case. Resulting offset is 0-- ehhh, not really...

It's only when BOTH subdivision lists are open that the final list needs a bunch of Fancy Shit done to it.

### I'm using the wrong representation

What if I convert this to another representation, and then solve the problem in that representation, then convert back?

I need a representation that's similar to what I do when concatenating onset functions. In that case, I simply evaluate all values in a range; that being the duration of the pattern. Can I do something similar here? Because this technique I'm exploring is VERY contrived.

Well, let's take a subdiv tree like `[[4,3],[2,2,2,1],[1,1,1,1,1,1,1]]`.

If every single position in our time grid has a value, why not just write it out like a list of values? You can represent the highest subdivision level with the highest number, and the lowest level with 0.

This becomes way simpler to represent. Behold: `[2,0,1,0,2,0,1]`!

Whoa! I've already had this idea floating around in my head weeks/months ago, actually. I think this is just a much simpler representation! Way, way simpler, infact!

### Okay, the revised approach

I think I will revise the entire velocity function class.

Instead of a subdivision tree, I will simply have a stress pattern. Then there is no need for an offset value - I simply have to rotate the pattern. concatenating functions also becomes trivial - you just concatenate the two patterns. So fucking simple. 

Fuck writing `VelocityFunc([[4,3],[2,2,2,1],[1,1,1,1,1,1,1]], resolution=Mixed("1/4"), velocities=[...], offset=Mixed("1/2"))`. You can instead just declare it like `VelocityFunc([1,0,2,0,1,2,0], resolution=Mixed("1/4")`. I'm still undecided on whether the velocities should be stored as state, or if they should be passed as arguments when evaluated. If the latter, then the objects themselves definitely become way simpler. But evaluation becomes like, `vfunc(t=Mixed("1 1/2"), velocities=[Mixed(x) for x in ["1/3", "2/3", 1]])`. 

Things like concatenation become simpler, too. Because if you store velocities in state, then concatenating two velocity functions requires you to decide on some opinionated behavior for handling difference in subdivision depth as well as figuring out which set of velocities to keep. But if you just pass velocities as an argument? You don't have to worry about *any* of that. Two different subdivision depths? Who cares, just concatenate the patterns. Before the pattern only had numbers 0 through 2, now there are some 3's as well. Great. That just means you better pass it lists with 4 different velocities.

For these reasons alone, it seems clear to me what the answer is. The only state this class neds is a pattern and a resolution.

It's also crossed my mind that even resolution could be passed as an argument. But I have reasons not to do that. Namely that there are useful methods like stretch, stretch_to_res, stretch_to_dur and truncate that either reference the resolution or modify it in some meaningful way. If I were to remove time resolution from state, then I'd have to delegate it to another class. Which honestly, I just don't see why I should do, to be honest? So for that reason, I'll keep the resolution in state.

# I need to go to bed

I'm almost done updating the VelocityFunc class. The only thing I need to do is implement the new truncate and concatenate methods. Luckily, this will be way easier than my previous approaches. Good night.